In [ ]:
#!pip install redis
#!sudo snap install redis

In [24]:
import pandas as pd
import numpy as np
import redis
import concurrent.futures
import pickle as pkl
from loguru import logger
import random
import string

# Sample datasets

In [97]:
def random_string(length):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

def create_random_df():
    unique_strings = [random_string(10) for i in range(1, 100)]
    data = {'col1': np.random.choice(unique_strings, 10),
            'col2': np.random.normal(100,10,10),
            'col3': np.random.choice(['A','B','C','D'],10)}
    return pd.DataFrame(data)
df=create_random_df()
df1=create_random_df()

In [98]:
df

,col1,col2,col3
0,coEvnluuNT,101.166692,C
1,JgoRGvhMTX,94.258721,D
2,RchPMPBskj,86.799792,B
3,oTIIRSQeKb,92.425227,A
4,oxtgmVTTyh,100.691178,D
5,kyAyznfqqV,117.567837,B
6,pgwglhogWJ,98.950190,D
7,WmAVcrAonK,104.296631,D
8,agnVotqonr,103.781794,D
9,UKLnKKDDUs,87.255390,B


In [99]:
df1

,col1,col2,col3
0,vIzrAxMDIx,98.010808,A
1,RhEJnLAFoi,102.010487,B
2,vEcZjtBzLD,112.437672,D
3,BBlswaYvji,108.277743,D
4,isebCUDsgh,116.039990,A
5,ASydNKfEOk,93.984055,D
6,vyMOBsdDcG,96.337740,B
7,OewKjylHtr,99.585763,D
8,VnImfPVUKN,109.421772,D
9,snWdlYcOJv,95.542644,B


# SAVE to Redis / GET from Redis dataframes

In [100]:
USE_CACHE = True

logger.info("Connect to redis")
r = redis.Redis(host="127.0.0.1", port=6379, db=1)

if r.ping():
    logger.info("Connecting to redis works!")
else:
    logger.error("Problem connect to redis")


def cache(key, use_cache=USE_CACHE):
    def def_decorator(func):
        def wrap(*args, **kwargs):
            overwrite_cache = kwargs.get("overwrite_cache", False)
            logger.info(overwrite_cache)
            if False == overwrite_cache and use_cache:
                buff = r.get(key)
                if buff:
                    return pkl.loads(buff)
                    logger.info("OK load cache for {}".format(key))
            result = func(*args)
            logger.info("OK load original for {}".format(key))
            if (False == use_cache) or (use_cache and overwrite_cache) or (not buff):
                r.set(key, pkl.dumps(result, pkl.HIGHEST_PROTOCOL))
                logger.info("OK save cache for {}".format(key))
            return result
        return wrap
    return def_decorator

def list_key(str_find):
    k=[]
    for key in r.scan_iter(str_find):
        k.append(str(key))
    return k
    
def get_cache_on_key(key):
    buff = r.get(key)
    if buff:
        return pkl.loads(buff)
    else:
        logger.info('Not found data in redis')

2023-02-05 11:05:56.076 | INFO     | __main__:<module>:3 - Connect to redis
2023-02-05 11:05:56.080 | INFO     | __main__:<module>:7 - Connecting to redis works!


In [101]:
@cache(key="df.test.1")
def __get_day_sales(df, overwrite_cache=False):
    return df

In [102]:
# save to redis
__get_day_sales(df, overwrite_cache=True)

2023-02-05 11:05:58.428 | INFO     | __main__:wrap:16 - True
2023-02-05 11:05:58.429 | INFO     | __main__:wrap:23 - OK load original for df.test.1
2023-02-05 11:05:58.431 | INFO     | __main__:wrap:26 - OK save cache for df.test.1


,col1,col2,col3
0,coEvnluuNT,101.166692,C
1,JgoRGvhMTX,94.258721,D
2,RchPMPBskj,86.799792,B
3,oTIIRSQeKb,92.425227,A
4,oxtgmVTTyh,100.691178,D
5,kyAyznfqqV,117.567837,B
6,pgwglhogWJ,98.950190,D
7,WmAVcrAonK,104.296631,D
8,agnVotqonr,103.781794,D
9,UKLnKKDDUs,87.255390,B


In [103]:
# list key in redis
str_find="df*"
list_key(str_find)

["b'df.test.1'"]

In [104]:
# get from redis
key="df.test.1"
df2=get_cache_on_key(key)
df2

,col1,col2,col3
0,coEvnluuNT,101.166692,C
1,JgoRGvhMTX,94.258721,D
2,RchPMPBskj,86.799792,B
3,oTIIRSQeKb,92.425227,A
4,oxtgmVTTyh,100.691178,D
5,kyAyznfqqV,117.567837,B
6,pgwglhogWJ,98.950190,D
7,WmAVcrAonK,104.296631,D
8,agnVotqonr,103.781794,D
9,UKLnKKDDUs,87.255390,B


In [105]:
# don't set to redis
__get_day_sales(df1, overwrite_cache=False)
# get on key
key="df.test.1"
df2=get_cache_on_key(key)
df2

2023-02-05 11:06:02.274 | INFO     | __main__:wrap:16 - False


,col1,col2,col3
0,coEvnluuNT,101.166692,C
1,JgoRGvhMTX,94.258721,D
2,RchPMPBskj,86.799792,B
3,oTIIRSQeKb,92.425227,A
4,oxtgmVTTyh,100.691178,D
5,kyAyznfqqV,117.567837,B
6,pgwglhogWJ,98.950190,D
7,WmAVcrAonK,104.296631,D
8,agnVotqonr,103.781794,D
9,UKLnKKDDUs,87.255390,B


In [106]:
# save to redis
__get_day_sales(df1, overwrite_cache=True)
# get on key
key="df.test.1"
df2=get_cache_on_key(key)
df2

2023-02-05 11:06:04.296 | INFO     | __main__:wrap:16 - True
2023-02-05 11:06:04.298 | INFO     | __main__:wrap:23 - OK load original for df.test.1
2023-02-05 11:06:04.299 | INFO     | __main__:wrap:26 - OK save cache for df.test.1


,col1,col2,col3
0,vIzrAxMDIx,98.010808,A
1,RhEJnLAFoi,102.010487,B
2,vEcZjtBzLD,112.437672,D
3,BBlswaYvji,108.277743,D
4,isebCUDsgh,116.039990,A
5,ASydNKfEOk,93.984055,D
6,vyMOBsdDcG,96.337740,B
7,OewKjylHtr,99.585763,D
8,VnImfPVUKN,109.421772,D
9,snWdlYcOJv,95.542644,B


# SAVE to Redis / GET from Redis pair "Key:Values"

In [117]:
%%time
def create_random_big_df():
    unique_strings = [random_string(10) for i in range(1, 100000)]
    data = {'col1': np.random.choice(unique_strings, 10000),
            'col2': np.random.normal(100,10,10000),
            'col3': np.random.choice(['A','B','C','D'],10000)}
    return pd.DataFrame(data)
df2=create_random_big_df()
print(df2.shape)
df2.head()

(10000, 3)
CPU times: user 1.4 s, sys: 0 ns, total: 1.4 s
Wall time: 1.44 s


,col1,col2,col3
0,WqQdvDaHyA,106.851814,B
1,ykkOaWZQYi,102.909115,D
2,BbBpSxLSZx,103.522302,A
3,ogzQNDTvSq,118.940135,C
4,juZdIBAiti,121.724189,B


In [118]:
# save to redis each pair
r2 = redis.Redis(host="127.0.0.1", port=6379, db=0)
def run_in_pipe(df):
    pipe = r2.pipeline()
    df_dict = df.to_dict()
    for col2, col3, col1 in zip(df_dict["col2"].values(), df_dict["col3"].values(), df_dict["col1"].values()):
        pipe.set(str(col1),f"{col3}_{col2}")
    return pipe.execute()

In [119]:
# delete all keys
r2.flushall()

True

In [120]:
%time _ = run_in_pipe(df2) # save to redis

CPU times: user 214 ms, sys: 0 ns, total: 214 ms
Wall time: 251 ms


In [121]:
# get from redis
def find_id(data):
    list_id = []
    if data:
        list_id = [x.decode('UTF-8') for x in set(r2.mget(data)) if x]
    return list_id

def get_from_redis(vals, max_workers=200):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        return [x for x in executor.map(lambda x: find_id(x), vals)]

In [123]:
%time df2['col_from_redis'] = get_from_redis(df2["col1"].map(lambda x: [x]))

CPU times: user 3.04 s, sys: 1.43 s, total: 4.47 s
Wall time: 5.02 s


In [125]:
df2.head()

,col1,col2,col3,col_from_redis
0,WqQdvDaHyA,106.851814,B,[B_106.85181431567248]
1,ykkOaWZQYi,102.909115,D,[D_102.9091146093156]
2,BbBpSxLSZx,103.522302,A,[A_103.52230211092746]
3,ogzQNDTvSq,118.940135,C,[C_118.94013488987596]
4,juZdIBAiti,121.724189,B,[B_121.72418868290654]
